# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [1]:
import json
import os
import logging
from typing import Sequence, List

import openai
from dotenv import load_dotenv
from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

level = logging.WARNING

logger = logging.getLogger(__name__)
logging.basicConfig(level=level)
logger.setLevel(level)
logging.getLogger("llama_index").setLevel(level)
logging.getLogger("asyncio").setLevel(level)

load_dotenv("../../../.env")
assert (key := os.getenv("OPENAI_API_KEY"))
openai.api_key = key
nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [4]:
from llama_index.agent import OpenAIAgent
from llama_index.agent.experimental_openai_agent import ExperimentalOpenAIAgent
from llama_index.llms import OpenAI

In [5]:
model = "gpt-3.5-turbo-0613"
#model = "gpt-4"
llm = OpenAI(model=model)
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
exp_agent = ExperimentalOpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

### Chat

In [6]:
response = exp_agent.chat("What is (121 * 3) + 42?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405
(121 * 3) + 42 is equal to 405.


In [7]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [9]:
response = await exp_agent.achat("What is 121 * 3?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363


121 multiplied by 3 is equal to 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [10]:
response = exp_agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a poem about a group of mice."
)

response_gen = response.response_gen

for ix, token in enumerate(response_gen):
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242
121 multiplied by 2 is equal to 242.

In a cozy little corner, where the moonlight softly gleams,
There lived a group of mice, with hopes and dreams.
With tiny paws and whiskers, they scurried to and fro,
Exploring every nook and cranny, where secrets would bestow.

With 242 little souls, they formed a joyful band,
Each mouse unique, with a story to understand.
They danced and played, their laughter filled the air,
A symphony of tiny voices, a melody so rare.

They built their homes with twigs and leaves, a sanctuary so grand,
A place to rest their weary feet, in a world so vast and grand.
They shared their tales of bravery, of adventures far and wide,
Of narrow escapes and friendships formed, with hearts open wide.

In the moonlit nights, they gathered around a tiny fire,
Sharing tales of love and hope, their spirits rising higher.
They dreamed of a world where mice could roam free,

### Async Streaming Chat

In [ ]:
response = await exp_agent.astream_chat(
    "What is 121 * 52 + 438? Once you have the answer, use that number to write a poem about a group of mice."
)

async for token in response.async_response_gen():
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 52
}
Got output: 6292
=== Calling Function ===
Calling function: add with args: {
  "a": 6292,
  "b": 438
}
Got output: 6730
121 multiplied by 52 is equal to 6292. Adding 438 to that gives us a total of 6730.

In a hidden corner of the world, where moonbeams softly glow,
A group of mice gathered, with hearts all aflow.
With 6730 tiny paws, they formed a mighty clan,
Exploring the unknown, with courage hand in hand.

Through fields of golden wheat, they scampered and they played,
Their tiny voices singing, a melody that never fades.
In their eyes, a spark of wonder, as they ventured far and wide,
Discovering the beauty of the world, with joy they couldn't hide.

They built their homes with care, in the hollows of old trees,
A sanctuary of love, where they could rest with ease.
With each passing day, their bond grew strong and true,
A family of mice, with dreams they would pursue.

In the moonlit nights, 

In [ ]:
raise Exception

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

In [ ]:
response = agent.chat("Tell me a story")
print(response)